In [1]:
import time
import folium
import geopandas as gpd #A more flexible package to work with geospatial data in python 
from itertools import product
import json
import matplotlib.pyplot as plt
import networkx as nx # networkx package 
import numpy as np
from osgeo import ogr #GDAL package 
import osmnx as ox 
import pandas as pd #Base package for data analysis and manipulation 
from pyproj import CRS #package for your projection management 
import random
import rasterio as rio
from rasterio import mask
import rasterstats as rs
import requests
import shapely.geometry #python package for basic spatial operation 
import xml.dom.minidom


### INTRODUCTION

The sections of this notebook are in order. 
The OSM settings should always be run.
Some of the operations in this notebook require data that can be downloaded from the data folder in this repository.

Downloading the AHN4 datasets can take quite some time, so make sure you have everything set up right before you do that. This also counts for downloading the networks.

The section with OSMOSIS can be ignored for now, but might be necessary for later if the downloading of network via OSMnx fails for some reason.

Adding grid value section is a work in progress.

### OSM settings

In [2]:
#adds surface as extra tag for the edges (THIS IS A REQUIRED STEP BEFORE YOU DOWNLOAD THE NETWORK)
ox.settings.useful_tags_way.extend(['surface','footway','cycleway', 'crossing', 'barrier'])
ox.settings.useful_tags_node.extend(['barrier'])

### NETWERK EXTENT 

In [4]:
#set the city names
g_namen = ['Amsterdam', 'Rotterdam', "Utrecht", "Den_Haag"]
#g_namen = ["Den_Haag"]

#download city geometries from file
project_gemeenten = gpd.read_file(r"data\boundaries\UrbanRunner_Areas.geojson") ## dit komt van de cbs gemeentegrenzen. Beschrijven bij data

#set additional geometry column of extent
project_gemeenten["extent"] = project_gemeenten.envelope.buffer(2000).envelope.to_crs("EPSG:4326")

#set index to gemeentenaam column
project_gemeenten.set_index('gemeentenaam', inplace=True)

#project to WGS84
project_gemeenten.to_crs("EPSG:4326", inplace=True)


In [ ]:
project_gemeenten.extent

##### network extent for testing

In [ ]:
Utrecht_centrum_extent = gpd.GeoSeries(shapely.geometry.Polygon([(135874.0,456545.4), (137381.2 ,456742.0), (137459.9 ,454749.8), (136018.1, 454703.9)]).envelope, crs= "EPSG:28992")

##### plot in folium

In [ ]:
map_centroid = project_gemeenten.unary_union.envelope.centroid.coords[0]

#create folium map
m = folium.Map(location = (map_centroid[1], map_centroid[0]))
#plot the convexhulls of each network region in the folium map
for _, r in project_gemeenten.iterrows():
    #unbuffered convex hull
    extent = gpd.GeoSeries(r["extent"])
    extent_j = extent.to_json()
    extent_j = folium.GeoJson(data=extent_j,
                           style_function=lambda x: {'fillColor': 'blue'})
    folium.Popup(r.name).add_to(extent_j)
    extent_j.add_to(m)

    #orginal geometry
    geom = gpd.GeoSeries(r['geometry'])
    geom_j = geom.to_json()
    geom_j = folium.GeoJson(data=geom_j,
                           style_function=lambda x: {'fillColor': 'green'})
    folium.Popup(r.name).add_to(geom_j)
    geom_j.add_to(m)

m

### GETTING NETWORK

In [ ]:
#download network for first time
for g in g_namen: ## beschrijven dat er twee netwerken worden gepakt: van walk en van bike, en dat deze dan worden samengevoegd met de networkx compose functie. 
    print('starting with', g)
    globals()[f"{g}_network_walk"] = ox.graph_from_polygon(project_gemeenten.loc[g].extent, network_type="walk", retain_all=True ,simplify=False) ## we hebben gekozen voor simplify false. Beschrijven wat dit inhoudt ahv osmnx documentation. We hebben de dijkstra algoritme getest op twee netwerken, één met simplify aan en de ander met simplify uit. Er was wel een verschil, maar niet groot genoeg om simplify te gebruiken. Retain_all= true moet ook worden uitgelegd waarom. Dit is namelijk omdat we twee verschillende netwerken samenvoegen, en dat we daarna de niet verbonden edges er afgooien.
    print('done with walking graph of', g)
    globals()[f"{g}_network_bike"] = ox.graph_from_polygon(project_gemeenten.loc[g].extent, network_type="bike", retain_all=True, simplify=False)
    print('done with biking graph of', g)
    globals()[f"{g}_network_both"] = nx.compose(globals()[f"{g}_network_walk"], globals()[f"{g}_network_bike"])
    globals()[f"{g}_network_both"] = ox.utils_graph.get_largest_component(globals()[f"{g}_network_both"])

##### NETWORK FROM GEOPACKAGE

In [ ]:
def graphs_from_geopackage(gemeente, type):
    """
    gemeente (str): name of gemeente
    type (str): 'gdf'  
                'graph'
                'both' 

    return: MultiDiGraph AND/OR nodes and edges (as gdf)
    """  
    fp = 'data/graphs/reprojected/'+gemeente+'_Graph.gpkg'
    gdf_nodes = gpd.read_file(fp, layer='nodes').set_index('osmid')
    gdf_edges = gpd.read_file(fp, layer='edges').set_index(['u', 'v', 'key'])
    assert gdf_nodes.index.is_unique and gdf_edges.index.is_unique
    if type == 'gdf':
        return gdf_nodes, gdf_edges
    elif type == 'graph':
        # convert the node/edge GeoDataFrames to a MultiDiGraph
        graph_attrs = {'crs': 'epsg:4326', 'simplified': False}
        G = ox.graph_from_gdfs(gdf_nodes, gdf_edges, graph_attrs)
        return G
    elif type == 'both':
        # convert the node/edge GeoDataFrames to a MultiDiGraph
        graph_attrs = {'crs': 'epsg:4326', 'simplified': False}
        G = ox.graph_from_gdfs(gdf_nodes, gdf_edges, graph_attrs)
        return G, gdf_nodes, gdf_edges

##### for testing (small network extent within Utrecht)

In [ ]:
#small network for testing
# simplify True
for g in ["Utrecht"]:
    globals()[f"{g}_network_walk_test"] = ox.graph_from_polygon(project_gemeenten.loc[g].extent, network_type="walk")
    print('done with walking network')
    globals()[f"{g}_network_bike_test"] = ox.graph_from_polygon(project_gemeenten.loc[g].extent, network_type="bike")
    print('done with biking network')
    globals()[f"{g}_network_both_test_simplified"] = nx.compose(globals()[f"{g}_network_walk_test"], globals()[f"{g}_network_bike_test"])

#simplify false
for g in ["Utrecht"]:
    globals()[f"{g}_network_walk_test"] = ox.graph_from_polygon(project_gemeenten.loc[g].extent, network_type="walk", simplify=False)
    print('done with walking network')
    globals()[f"{g}_network_bike_test"] = ox.graph_from_polygon(project_gemeenten.loc[g].extent, network_type="bike", simplify=False)
    print('done with biking network')
    globals()[f"{g}_network_both_test"] = nx.compose(globals()[f"{g}_network_walk_test"], globals()[f"{g}_network_bike_test"])

##### testing with small extent

In [ ]:
smallextent_walk = ox.graph_from_polygon(Utrecht_centrum_extent.to_crs("EPSG:4326")[0], network_type="walk")
print('done with walking network')
smallextent_bike = ox.graph_from_polygon(Utrecht_centrum_extent.to_crs("EPSG:4326")[0], network_type="bike")
print('done with biking network')
smallextent_both = nx.compose(smallextent_walk, smallextent_bike)

In [ ]:
smallextent_edges = ox.graph_to_gdfs(smallextent_both, nodes=False)

### DOWNLOADING AHN, ADDING NODE ELEVATION, ADDING EDGE GRADES

#### DOWNLOADING AHN

In [ ]:
ahn_datavlakken = gpd.read_file(r"data/kaartbladen_AHN4.gpkg") ## AHN4 data beschrijven in methode. Kaartbladen_AHN4 bestaan uit tiles met download links naar verschillende varianten van AHN4. Wij gebruiken AHN4_0.5M_DTM. Beschrijven waarom deze wordt gepakt.
ahn_datavlakken.to_crs("EPSG:4326", inplace=True)
for g in g_namen:
    globals()[f"{g}_ahnvlakken"]=ahn_datavlakken.clip(project_gemeenten.loc[g].extent) ## Beschrijven in de methode hoe de juiste tiles wordt gekozen.

In [ ]:
for gemeente in g_namen:
    globals()[f"{gemeente}_ahnvlakken"]=ahn_datavlakken.clip(project_gemeenten.loc[gemeente].extent)
    for i in range(globals()[f"{gemeente}_ahnvlakken"].Name_1.count()):    
        name = globals()[f"{gemeente}_ahnvlakken"].iat[i,2]

        response = requests.get(globals()[f"{gemeente}_ahnvlakken"].iat[i,3])
        print('done with downloading number ' + i + ' of gemeente ' + gemeente)
        open('C:\\Users\\danny\\Documents\\persoonlijk\\GIMA\\modules\\module 6\\data\\AHN4_05M_DTM\\'+gemeente + '\\'+ name + '.zip', "wb").write(response.content)

#### ADDING NODE ELEVATION

In [ ]:
for g in g_namen:
    globals()[f"{g}_DTM_list"] = []
    for i in range(globals()[f"{g}_ahnvlakken"].Name_1.count()):    
        name = globals()[f"{g}_ahnvlakken"].iat[i,2]
        globals()[f"{g}_DTM_list"].append('C:\\Users\\danny\\Documents\\persoonlijk\\GIMA\\modules\\module 6\\data\\AHN4_05M_DTM\\'+ g + '\\M_'+ name +".tif")

In [ ]:
for g in g_namen:
    globals()[f"{g}_network_both"] = ox.project_graph(globals()[f"{g}_network_both"], to_crs="epsg:28992")
    globals()[f"{g}_network_both"] = ox.add_node_elevations_raster(globals()[f"{g}_network_both"], globals()[f"{g}_DTM_list"])
    globals()[f"{g}_nodes"], globals()[f"{g}_edges"] = ox.graph_to_gdfs(globals()[f"{g}_network_both"])

### ADDING HEAT DATA

In [ ]:
def EnrichEdgesWithRasterInfo(edges, raster, statsprefix):
    edges['UID'] = range(0, len(edges)) # add a temporary Unique ID column
    e = edges.loc[:, ['UID', 'geometry']] # make subset of relevant columns
    e_zonalstats = rs.zonal_stats(e, raster, prefix=statsprefix, geojson_out=True) # perform spatial overlay/ zonal statistics and add statistics
    e_props = pd.DataFrame.from_dict(e_zonalstats).properties # convert to dataframe and select only the properties (results again in dictionary)
    e_propsdf = pd.DataFrame.from_dict(list(e_props)) # convert dictionary with properties to a pandas dataframe
    edges_updated = edges.join(other=e_propsdf.set_index('UID'), on=('UID')) # join stats to the original edges and save as updated edges
    return edges_updated

In [ ]:
# Preprocess UHIi map
rastfile = rio.open('data/RIVM_R88_20170621_gm_actueelUHI.tif')
for g in g_namen:   
    bboxshape = [json.loads(gpd.GeoDataFrame({"geometry": project_gemeenten.loc[g].extent.buffer(0.1).envelope}, 
        index=[0], crs="EPSG:4326").to_crs("EPSG:28992").to_json())['features'][0]['geometry']]
    UHImap, UHImap_transform = rio.mask.mask(rastfile, shapes=bboxshape, crop=True)
    with rio.open(f"{g}_hittedata.tif", 'w', driver="GTiff",
                   height=UHImap.shape[1], width=UHImap.shape[2], 
                   transform=UHImap_transform, crs=CRS.from_epsg(28992),
                   nodata=255, dtype='float32', count= 1) as file:
        file.write(UHImap)

In [ ]:
#PLOTTING MAP
UHI = rio.open('utrecht_hittedata.tif').read(1, masked=True)
fig, ax = plt.subplots(figsize=(20,20))
UHIplot = ax.imshow(UHI, cmap='jet')
ax.set_title("UHI map of area", fontsize=25)
cbar = fig.colorbar(UHIplot, fraction=0.035, pad=0.01)
cbar.ax.get_yaxis().labelpad = 15
cbar.ax.set_ylabel('physiologocal equivalent temperature (°C)', rotation=270)
ax.set_axis_off()
plt.show()

In [ ]:
for g in g_namen:
    globals()[f"{g}_edges"] = EnrichEdgesWithRasterInfo(globals()[f"{g}_edges"], f"{g}_hittedata.tif", 'UHI_')

### FINISHING SLOPE DATA

#### DOWNLOADING NETWORK AND REGRAPHING

The nodes are downloaded to bring into QGIS. In QGIS the nodes with the missing elevation is interpolated.

In [ ]:
for g in g_namen:
    globals()[f"{g}_nodes"].to_file(f'data/graphs/retry/{g}_graph.gpkg', driver = "GPKG", layer= 'nodes') 
    globals()[f"{g}_edges"].to_file(f'data/graphs/retry/{g}_graph.gpkg', driver = "GPKG", layer= 'edges') 

In QGIS the following steps are performed:
1. add nodes layer;
2. in field calculator run the expression for the added node layer:

        if(elevation is Null, array_mean(overlay_nearest(layer:='nodes', filter:= elevation is not Null, expression:= elevation, limit:=5), elevation)
3. save edits.



##### regraph, add edge grades, and get nodes and edges gdfs

In [5]:
for g in g_namen:
    globals()[f"{g}_nodes"] = gpd.read_file(f'data/graphs/retry/{g}_graph.gpkg', layer= 'nodes').set_index('osmid')
    globals()[f"{g}_edges"] = gpd.read_file(f'data/graphs/retry/{g}_graph.gpkg', layer= 'edges').set_index(['u','v','key'])
    globals()[f"{g}_network_both"] = ox.graph_from_gdfs(globals()[f"{g}_nodes"],globals()[f"{g}_edges"])
    globals()[f"{g}_network_both"] = ox.add_edge_grades(globals()[f"{g}_network_both"])
    globals()[f"{g}_nodes"], globals()[f"{g}_edges"] = ox.graph_to_gdfs(globals()[f"{g}_network_both"], nodes=True, edges=True, node_geometry=False, fill_edge_geometry=False)

### GREENSPACES, WATERTAPS AND OBSTACLES

In [6]:
for g in g_namen:
    globals()[f"greenspaces_{g}"] = ox.geometries_from_polygon(project_gemeenten.loc[g].extent, tags={'landuse':['village_green','recreation_ground','grass','forest'], 'leisure':['nature_reserve','park','garden'],'natural':['fell','heath','wood','wetland','coastline','beach']}) ## beschrijven
    globals()[f"greenspaces_{g}"] = globals()[f"greenspaces_{g}"].loc['way'][["geometry","natural","leisure","landuse"]].to_crs("EPSG:28992")
    globals()[f"greenspaces_{g}"].geometry = globals()[f"greenspaces_{g}"].buffer(5)
    #globals()[f"greenspaces_{g}"].to_crs("EPSG:4326", inplace=True) 

    globals()[f"waterpoints_{g}"] = ox.geometries_from_polygon(project_gemeenten.loc[g].extent, tags={'amenity':'drinking_water'})
    globals()[f"waterpoints_{g}"].to_crs('EPSG:28992', inplace=True)
    globals()[f"waterpoints_{g}"]['buffered'] = globals()[f"waterpoints_{g}"].buffer(50) ## dit moet worden onderbouwd: waarom 50 meter buffer? & in de discussie: watertappunten is lastig, want je wilt niet na 100m al langs een watertappunt lopen
    globals()[f"waterpoints_{g}"].set_geometry('buffered', inplace=True, crs="EPSG:28992")
    #globals()[f"waterpoints_{g}"].to_crs('EPSG:4326', inplace=True)
    globals()[f"waterpoints_{g}"] = globals()[f"waterpoints_{g}"][['amenity','buffered']]

    #globals()[f"obstruction_{g}"] = ox.geometries_from_polygon(project_gemeenten.loc[g].extent, tags={'barrier':['gate', 'stile'], 'footway':'crossing', 'cycleway':'crossing'}) ## barriers beschrijven, hiernaast wordt ook gebruik gemaakt van traffic signals, die al inherent in het netwerk is
    #globals()[f"obstruction_nodes_{g}"] = globals()[f"obstruction_{g}"].loc['node']['barrier'].reset_index()
    #globals()[f"obstruction_ways_{g}"] = globals()[f"obstruction_{g}"].loc['way'][['footway','cycleway']].reset_index()

c:\Users\danny\anaconda3\envs\ox\lib\site-packages\osmnx\geometries.py:805: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  for merged_outer_linestring in list(merged_outer_linestrings):
c:\Users\danny\anaconda3\envs\ox\lib\site-packages\osmnx\geometries.py:805: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for merged_outer_linestring in list(merged_outer_linestrings):
c:\Users\danny\anaconda3\envs\ox\lib\site-packages\osmnx\geometries.py:805: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  for merged_oute

##### adding to node and edges

In [31]:
for g in g_namen:
    globals()[f"{g}_edges"] = gpd.sjoin(globals()[f"{g}_edges"], globals()[f"greenspaces_{g}"], rsuffix = "greenspace", how = 'left')
    globals()[f"{g}_edges"] = gpd.sjoin(globals()[f"{g}_edges"], globals()[f"waterpoints_{g}"], rsuffix = "waterpoints", how = 'left')
    nodes_disruptions = globals()[f"{g}_nodes"][['highway', 'barrier']]
    globals()[f"{g}_edges"] = globals()[f"{g}_edges"].join(nodes_disruptions.rename_axis('u'), how = 'left', rsuffix = "_obstnodes_u")
    globals()[f"{g}_edges"] = globals()[f"{g}_edges"].join(nodes_disruptions.rename_axis('v'), how = 'left', rsuffix = "_obstnodes_v")


### SAVING NETWORKS TO GEOPACKAGE

In [ ]:
#saving networks
def saving_networks(network,g):
    ox.save_graph_geopackage(network, filepath='data\\graphs\\' + g + '_Graph_complete2.gpkg')

In [ ]:
for g in g_namen:
    saving_networks(test_Utrecht_graph, g = g)

### CHECKING SOME OF THE VALUES

##### CHECKING UNIQUE VALUE

In [32]:
Utrecht_edges.columns

Index(['osmid', 'highway', 'oneway', 'length', 'UID', 'UHI_min', 'UHI_max',
       'UHI_mean', 'UHI_count', 'geometry', 'grade', 'grade_abs', 'surface',
       'name', 'maxspeed', 'bridge', 'width', 'service', 'lanes', 'access',
       'cycleway', 'tunnel', 'footway', 'junction', 'crossing', 'est_width',
       'ref', 'highway_obstnodes_u', 'barrier', 'highway_obstnodes_v',
       'barrier_obstnodes_v', 'index_greenspace', 'natural', 'leisure',
       'landuse', 'index_waterpoints0', 'index_waterpoints1', 'amenity'],
      dtype='object')

In [28]:
attributes = ['UHI_mean', 'barrier', 'highway', 'cycleway', 'footway', 'surface', 'highway_obstnodes_u','barrier']
for v in attributes:
    globals()[f"{v}_uniques"] = []
    for g in g_namen:
        globals()[f"{v}_uniques"].extend(globals()[f"{g}_edges"][v].unique())
    globals()[f"{v}_uniques"] = list(set(globals()[f"{v}_uniques"]))

In [29]:
print(highway_obstnodes_u_uniques)

['crossing', 'elevator', 'emergency_bay', 'traffic_signals', 'bus_stop', 'turning_loop', 'speed_camera', 'construction', 'give_way', 'steps', 'mini_roundabout', 'turning_circle', nan, 'stop', 'trailhead', 'passing_place', 'motorway_junction', 'milestone', 'street_lamp']


##### checking occurences of values

In [65]:
def occurence_value(value, variable):
    for g in g_namen:
        edges = globals()[f"{g}_edges"]
        print(value, 'occurs', len(edges[edges[variable] == value]), 'times for the variable', variable, 'in the area of', g)

In [70]:
occurence_value('rasin', 'surface')

rasin occurs 34 times for the variable surface in the area of Amsterdam
rasin occurs 0 times for the variable surface in the area of Rotterdam
rasin occurs 134 times for the variable surface in the area of Utrecht
rasin occurs 0 times for the variable surface in the area of Den_Haag


In [71]:
Utrecht_edges[Utrecht_edges['surface'] == 'rasin']

osmid   highway  oneway  length     UID  \
u          v          key                                                
3586879119 3586835777 0    352900663      path   False  11.798   24298   
3586835777 3586879119 0    352900663      path   False  11.798   25380   
2653000876 2653000878 0    259843397  cycleway   False   9.571   65564   
2653000878 2653000876 0    259843397  cycleway   False   9.571   65566   
2652564691 2652564689 0    259844573   footway   False   7.766   67973   
...                              ...       ...     ...     ...     ...   
2652632796 2652632800 0    259844369   footway   False   6.230  337568   
2652632800 2652632796 0    259844369   footway   False   6.230  337570   
                      0    259844369   footway   False   6.230  337570   
           2652632803 0    259844369   footway   False   2.377  337571   
                      0    259844369   footway   False   2.377  337571   

                           UHI_min  UHI_max  UHI_mean  UHI_count  \
u          v          key                                          
3586879119 3586835777 0      1.122    1.143    1.1325          2   
3586835777 3586879119 0      1.122    1.143    1.1325          2   
2653000876 2653000878 0      0.787    0.861    0.8240          2   
2653000878 2653000876 0      0.787    0.861    0.8240          2   
2652564691 2652564689 0      0.918    0.928    0.9230          2   
...                            ...      ...       ...        ...   
2652632796 2652632800 0      0.846    0.846    0.8460          2   
2652632800 2652632796 0      0.846    0.846    0.8460          2   
                      0      0.846    0.846    0.8460          2   
           2652632803 0      0.846    0.846    0.8460          1   
                      0      0.846    0.846    0.8460          1   

                                                                    geometry  \
u          v          key                                                      
3586879119 3586835777 0    LINESTRING (134632.653 447922.728, 134644.463 ...   
3586835777 3586879119 0    LINESTRING (134644.463 447923.492, 134632.653 ...   
2653000876 2653000878 0    LINESTRING (133113.981 449941.266, 133105.035 ...   
2653000878 2653000876 0    LINESTRING (133105.035 449944.744, 133113.981 ...   
2652564691 2652564689 0    LINESTRING (133402.593 450076.067, 133410.090 ...   
...                                                                      ...   
2652632796 2652632800 0    LINESTRING (133300.736 450169.100, 133296.013 ...   
2652632800 2652632796 0    LINESTRING (133296.013 450173.181, 133300.736 ...   
                      0    LINESTRING (133296.013 450173.181, 133300.736 ...   
           2652632803 0    LINESTRING (133296.013 450173.181, 133293.987 ...   
                      0    LINESTRING (133296.013 450173.181, 133293.987 ...   

                           ...  barrier  highway_obstnodes_v  \
u          v          key  ...                                 
3586879119 3586835777 0    ...      NaN                  NaN   
3586835777 3586879119 0    ...      NaN                  NaN   
2653000876 2653000878 0    ...      NaN                  NaN   
2653000878 2653000876 0    ...      NaN                  NaN   
2652564691 2652564689 0    ...      NaN                  NaN   
...                        ...      ...                  ...   
2652632796 2652632800 0    ...      NaN                  NaN   
2652632800 2652632796 0    ...      NaN                  NaN   
                      0    ...      NaN                  NaN   
           2652632803 0    ...      NaN                  NaN   
                      0    ...      NaN                  NaN   

                          barrier_obstnodes_v index_greenspace natural  \
u          v          key                                                
3586879119 3586835777 0                   NaN              NaN     NaN   
3586835777 3586879119 0                   NaN              NaN     NaN   
2653000876 26

### ROUTES

In [54]:
def get_random_XY_in_polygon(poly): ## manier om XY coordinaten te pakken binnen een polygon
    minx, miny, maxx, maxy = poly.bounds
    while True:
        p = shapely.geometry.Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
        if poly.contains(p):
            return p.coords[0][0], p.coords[0][1]

In [55]:
#number of nodes
n_nodes = 100

#getting random nodes
for g in g_namen:
    X_list = []
    Y_list = []    
    for i in range(n_nodes):
        x, y = get_random_XY_in_polygon(project_gemeenten.to_crs('epsg:28992').loc[g].geometry)
        X_list.append(x)
        Y_list.append(y)
    globals()[f"{g}_random_nodes"] = ox.nearest_nodes(Utrecht_network_both, X_list, Y_list)

In [56]:
nx.dijkstra_path(G=Utrecht_network_both, source=Utrecht_random_nodes[3], target=Utrecht_random_nodes[1], weight='final_cost')

[253378683,
 2611107138,
 2611107136,
 253378685,
 253378686,
 1234560793,
 2450817834,
 1763475769,
 1234566855,
 45295364,
 45293771,
 45289709,
 45288171,
 4568726751,
 45287972,
 45287985,
 45285608,
 45284876,
 45284396,
 45283823,
 2725252322,
 45278729,
 45272633,
 45272137,
 2725252320,
 45270457,
 45271743,
 8988292690,
 9077584221,
 9077584226,
 9077584222,
 45254992,
 45251576,
 45245350,
 45244434,
 45243995,
 535259339,
 45243364,
 248999802,
 45241236,
 45241210,
 8942646649,
 8942706790,
 8942706791,
 8942706792,
 8942706793,
 8942706794,
 8942706795,
 8942706796,
 8942706797,
 8942706798,
 8942706799,
 8942646654,
 2888795019,
 45241272,
 45239145,
 2888795006,
 332124150,
 6788184961,
 45232965,
 45231980,
 352863947,
 7624140911,
 352863948,
 352863949,
 307512495,
 307512494,
 307512493,
 4990848633,
 4990848632,
 4990848631,
 5001532074,
 4990848630,
 4990848629,
 5868331580,
 4532425339,
 5868331549,
 4532425340,
 6988416965,
 7196306210,
 5868331545,
 9263044236,


In [ ]:
#getting nodes from the random XY sets
#UNSIMPLIFIED

source_nodes = globals()[f"{g}_random_nodes"][0][:50]
goal_nodes = globals()[f"{g}_random_nodes"][0][50:100]

In [ ]:
random_shortest_routes = ox.distance.shortest_path(
    test_Utrecht_graph, source_nodes, goal_nodes, weight='final_cost', cpus=1)

In [ ]:
random_shortest_routes

##### test: comparing simplified with nonsimplified

In [ ]:
for g in ["Utrecht"]:
    X_list = []
    Y_list = []    
    for i in range(100):
        x, y = get_random_XY_in_polygon(project_gemeenten.loc[g].geometry)
        X_list.append(x)
        Y_list.append(y)
    #random_nodes = ox.nearest_nodes(, X_list, Y_list, return_dist=True)

In [ ]:
#getting nodes from the random XY sets
#UNSIMPLIFIED
random_nodes = ox.nearest_nodes(u_network_both, X_list, Y_list, return_dist=True)
source_nodes = random_nodes[0][:50]
goal_nodes = random_nodes[0][50:100]

In [ ]:
#getting nodes from the random XY sets
#UNSIMPLIFIED
Utrecht_random_nodes = ox.nearest_nodes(Utrecht_network_both_test, X_list, Y_list, return_dist=True)
Utrecht_source_nodes = Utrecht_random_nodes[0][:50]
Utrecht_goal_nodes   =    Utrecht_random_nodes[0][50:100]
#SIMPLIFIED
Utrecht_random_nodes_simplified = ox.nearest_nodes(Utrecht_network_both_test_simplified, X_list, Y_list, return_dist=True)
Utrecht_source_nodes_simplified = Utrecht_random_nodes_simplified[0][:50]
Utrecht_goal_nodes_simplified   =   Utrecht_random_nodes_simplified[0][50:100]

In [ ]:
# calculating shortest route of the source and goal nodes
#UNSIMPLIFIED
start_time = time.time()
random_shortest_routes = ox.distance.shortest_path(
    Utrecht_network_both_test, Utrecht_source_nodes, Utrecht_goal_nodes, weight='length', cpus=1)

print("gathering random routes for unsimplified network takes %s seconds" % (time.time() - start_time))
randomroute_time = (time.time() - start_time)

#SIMPLIFIED
start_time = time.time()
random_shortest_routes_simplified = ox.distance.shortest_path(
    Utrecht_network_both_test_simplified, Utrecht_source_nodes_simplified, Utrecht_goal_nodes_simplified, weight='length', cpus=1)
    
print("gathering random routes for simplified network takes %s seconds" % (time.time() - start_time))
randomroute_time_simplified = (time.time() - start_time)

In [ ]:
ox.folium.plot_route_folium(u_network_both, random_shortest_routes[2], route_map=None, popup_attribute=None, tiles='cartodbpositron', zoom=1, fit_bounds=True)

In [ ]:
ox.folium.plot_route_folium(u_network_both_simplified, random_shortest_routes_simplified[2], route_map=None, popup_attribute=None, tiles='cartodbpositron', zoom=1, fit_bounds=True)

### USING OSMOSIS

#### for ways

#### for greenspaces


COUNTING greenspace per type

In [ ]:
doc = xml.dom.minidom.parse(r"data/OSM/utrecht_greenspaces.osm")

ways = doc.getElementsByTagName('way')

In [ ]:
wayWithLanduse = []
wayWithLeisure = []
wayWithNatural = []
for way in ways:
    for way_child in way.childNodes:
        if type(way_child) == xml.dom.minidom.Text:
            continue
        elif way_child.hasAttribute('k'):
            if way_child.getAttribute('k') == 'landuse': 
                wayWithLanduse.append(way)
            elif way_child.getAttribute('k') == 'leisure':
                wayWithLeisure.append(way)
            elif way_child.getAttribute('k') == 'natural':
                wayWithNatural.append(way)

In [ ]:
print(len(wayWithLeisure), len(wayWithLanduse), len(wayWithNatural))


### COST FUNCTION

In [ ]:
Utrecht_nodes, Utrecht_edges = graphs_from_geopackage('Utrecht', 'gdf')

In [ ]:
Utrecht_edges_dropped = Utrecht_edges.drop(columns=["lanes","width","est_width", "ref",'index_waterpoints0', 'index_waterpoints1','UID','maxspeed', 'bridge', 'access'])

##### SAVE GDFS WITHOUT COST COLUMNS TO GEOPACKAGE
These will be baseline edge and nodes that will be reused for different personas

In [52]:
for g in g_namen: 
    globals()[f"{g}_edges"].to_file(f'data/graphs/retry/baseline_edges/{g}_edges_baseline.gpkg', driver = "GPKG", layer= 'edges') 

c:\Users\danny\anaconda3\envs\ox\lib\site-packages\geopandas\io\file.py:389: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(gdal_version) >= LooseVersion("3.0.0") and crs:
c:\Users\danny\anaconda3\envs\ox\lib\site-packages\geopandas\io\file.py:389: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(gdal_version) >= LooseVersion("3.0.0") and crs:
c:\Users\danny\anaconda3\envs\ox\lib\site-packages\geopandas\io\file.py:389: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(gdal_version) >= LooseVersion("3.0.0") and crs:
c:\Users\danny\anaconda3\envs\ox\lib\site-packages\geopandas\io\file.py:389: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(gdal_version) >= LooseVersion("3.0.0") and crs:


In [53]:
Utrecht_network_both = ox.graph_from_gdfs(Utrecht_nodes, Utrecht_edges)

In [77]:
Amsterdam_edges.grade.max()

5.982

#### MULTIPLYING BY LENGTH

In [96]:
for g in g_namen:
    edges = globals()[f"{g}_edges"] 

    #waterpoints
    edges['cost_waterpoints'] = 1
    edges['cost_waterpoints'][edges.amenity == 'drinking_water'] = 0

    #UHI
    edges['cost_UHI'] = edges['UHI_mean'] / 2.808000087738037 #2.808 comes from the highest UHI_mean value of all the areas

    #Greenspace
    edges['cost_greenspace'] = 0
    edges['cost_greenspace'][(edges['landuse'] == '') & (edges['leisure'] == '') & (edges['natural'] == '')] = 1

    #slope
    edges['cost_grade'] = 0
    edges['cost_grade'][(edges['grade'] > 0)] = edges['grade'] / 5.982 #5.982 is the highest grade value of all the areas

    #disruptions
    edges['cost_disruptions'] = 0
    edges['cost_disruptions'][(edges['footway'] == 'crossing') |(edges['cycleway'] == 'crossing') | (edges['barrier'] == 'gate') |(edges['barrier'] == 'stile') ] = 0.5
    edges['cost_disruptions'][(edges['highway_obstnodes_u'] == 'traffic_signals') |(edges['highway_obstnodes_v'] == 'traffic_signals')] = 1
    
    #surface_unpaved
    edges['cost_surface_prefunpaved'] = 0.5
    edges['cost_surface_prefunpaved'][edges['surface'].isin(['dirt;grass','grass_paver','mud','dirt','unpaved','wetland','clay','grass','sand','ground','earth','woodchips','reed_bed','soil'])] = 0
    edges['cost_surface_prefunpaved'][edges['surface'].isin(['dirt;gravel','crushed_shells','gravel','fine_gravel','shells','schelpen','rubber'])] = 0.25
    edges['cost_surface_prefunpaved'][edges['surface'].isin(['compacted','wood','resin','rasin'])] = 0.75
    edges['cost_surface_prefunpaved'][edges['surface'].isin(['concrete-slabs','steel','paving_stones;asphalt','concrete:lanes','concrete_slab','concrete_slabs','concrete:deconstructed_czech_hedgehogs','paved','asphalt','metal','marble','concrete:plates','concrete:tiles','concrete','zinc','bricks','brick','tiles','concrete_tiles:30','paving_stones:20','paving_stones:30','metal_grid','asphalt;paving_stones:30x30','stepping_stones','paving_stones','stone','pebblestone','cobblestone','unhewn_cobblestone'])] = 1
    
    #surface_paved
    edges['cost_surface_prefpaved'] = 0.5
    edges['cost_surface_prefpaved'][edges['surface'].isin(['dirt;grass','grass_paver','mud','dirt','unpaved','wetland','clay','grass','sand','ground','earth','woodchips','reed_bed','soil'])] = 1
    edges['cost_surface_prefpaved'][edges['surface'].isin(['dirt;gravel','crushed_shells','gravel','fine_gravel','shells','schelpen','rubber'])] = 0.75
    edges['cost_surface_prefpaved'][edges['surface'].isin(['compacted','wood','resin','rasin'])] = 0.25
    edges['cost_surface_prefpaved'][edges['surface'].isin(['concrete-slabs','steel','paving_stones;asphalt','concrete:lanes','concrete_slab','concrete_slabs','concrete:deconstructed_czech_hedgehogs','paved','asphalt','metal','marble','concrete:plates','concrete:tiles','concrete','zinc','bricks','brick','tiles','concrete_tiles:30','paving_stones:20','paving_stones:30','metal_grid','asphalt;paving_stones:30x30','stepping_stones','paving_stones','stone','pebblestone','cobblestone','unhewn_cobblestone'])] = 0
    
    
    globals()[f"{g}_edges"] = edges
    

C:\Users\danny\AppData\Local\Temp/ipykernel_2908/3486008435.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edges['cost_surface_prefpaved'][edges['surface'].isin(['dirt;grass','grass_paver','mud','dirt','unpaved','wetland','clay','grass','sand','ground','earth','woodchips','reed_bed','soil'])] = 1
C:\Users\danny\AppData\Local\Temp/ipykernel_2908/3486008435.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edges['cost_surface_prefpaved'][edges['surface'].isin(['dirt;gravel','crushed_shells','gravel','fine_gravel','shells','schelpen','rubber'])] = 0.75
C:\Users\danny\AppData\Local\Temp/ipykernel_2908/3486008435.p

##### SAVE GRAPHS WITH ONLY COST COLUMNS, NO FINAL COST

In [98]:
for g in g_namen:
    globals()[f"{g}_edges"].to_file(f'data/graphs/retry/with_costs/{g}_edges_with_costs.gpkg', driver = "GPKG", layer= 'edges')

c:\Users\danny\anaconda3\envs\ox\lib\site-packages\geopandas\io\file.py:389: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(gdal_version) >= LooseVersion("3.0.0") and crs:
c:\Users\danny\anaconda3\envs\ox\lib\site-packages\geopandas\io\file.py:389: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(gdal_version) >= LooseVersion("3.0.0") and crs:
c:\Users\danny\anaconda3\envs\ox\lib\site-packages\geopandas\io\file.py:389: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(gdal_version) >= LooseVersion("3.0.0") and crs:
c:\Users\danny\anaconda3\envs\ox\lib\site-packages\geopandas\io\file.py:389: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(gdal_version) >= LooseVersion("3.0.0") and crs:


#### CALCULATING FINAL COST COLUMN

In [ ]:
for g in g_namen:
#final cost column
    edges['final_cost'] = ((0.3*edges['cost_waterpoints']) + (0.6*edges['cost_UHI']) + (0.8*edges['cost_greenspace']) + (0.2*edges['cost_grade']) + (0.3*edges['cost_disruptions']) + (0.1*edges['cost_surface'])) * edges['length'] 

In [ ]:
# get network
for g in g_namen:
    globals()[f"{g}_edges"] = gpd.read_file(f'data/graphs/') 

In [ ]:
gpd.read_file('data/')